# Import libraries

In [1]:
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

from scipy.stats import norm

# Load dataset

In [2]:
# Dataset name
dataset_name = 'sentence_dataset.csv'

# Getting path of current working directory
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Path to sentence_dataset.csv
sentence_dataset_path = os.path.normpath(os.path.join(current_directory, '..', 'datasets', dataset_name))
print(f"Path to sentence_dataset.csv: {sentence_dataset_path}")

Current directory: /Users/niclasgriesshaber/Desktop/guilds-llm/03_probit_regression
Path to sentence_dataset.csv: /Users/niclasgriesshaber/Desktop/guilds-llm/datasets/sentence_dataset.csv


In [3]:
# Load dataset
df = pd.read_csv(sentence_dataset_path)

In [4]:
# Check dataset
df.head()

,index,country,guild,century,year,entry_barriers,human_capital,product_quality,markets,enforcement,religion,other,dummy_mexico,dummy17,dummy18,mexico_x_17,mexico_x_18
0,1,mexico,cotton-weavers,18,1757,1,1,1,1,1,0,0,1,0,1,0,1
1,2,mexico,cotton-weavers,18,1757,0,1,1,1,1,0,0,1,0,1,0,1
2,3,mexico,cotton-weavers,18,1757,0,1,1,0,1,0,0,1,0,1,0,1
3,4,mexico,cotton-weavers,18,1757,0,1,1,0,1,0,0,1,0,1,0,1
4,5,mexico,cotton-weavers,18,1757,0,1,1,0,1,0,0,1,0,1,0,1


# Function to run probit regression for all categories

In [5]:
# List of dependent variables
dependent_vars = [
    'entry_barriers',
    'human_capital',
    'product_quality',
    'markets',
    'enforcement',
    'religion',
    'other'
]

# Ensure that dummy_mexico and century are categorical variables
df['dummy_mexico'] = df['dummy_mexico'].astype('category')
df['century'] = df['century'].astype(str)  # Ensure century is string for consistency
df['century'] = df['century'].astype('category')

# Open the text file for writing
with open('probit_regression_results.txt', 'w') as f:

    for dep_var in dependent_vars:
        # Fit the probit model with interaction terms and robust standard errors
        formula = f'{dep_var} ~ C(dummy_mexico) * C(century)'
        model = smf.probit(formula, data=df)
        result = model.fit(cov_type='HC3', disp=0)  # disp=0 suppresses the fit summary

        # Write the regression results to the file
        f.write(f"\nProbit Regression Results for '{dep_var}':\n")
        f.write(result.summary().as_text())
        f.write("\n")

        # Get parameters and covariance matrix
        params = result.params
        cov_params = result.cov_params()

        # Function to create exogenous variables for specified dummy_mexico and century
        def create_exog(dummy_mexico, century):
            exog = []
            # Intercept
            exog.append(1)
            # C(dummy_mexico)[T.1]
            exog.append(1 if dummy_mexico == 1 else 0)
            # C(century)[T.17]
            exog.append(1 if century == '17' else 0)
            # C(century)[T.18]
            exog.append(1 if century == '18' else 0)
            # Interaction terms
            exog.append(1 if (dummy_mexico == 1 and century == '17') else 0)
            exog.append(1 if (dummy_mexico == 1 and century == '18') else 0)
            return np.array(exog)

        # Function to compute marginal effect of dummy_mexico at a given century
        def compute_marginal_effect(century_var):
            exog0 = create_exog(0, century_var)
            exog1 = create_exog(1, century_var)
            xb0 = np.dot(exog0, params)
            xb1 = np.dot(exog1, params)
            prob0 = norm.cdf(xb0)
            prob1 = norm.cdf(xb1)
            me = prob1 - prob0
            # Compute gradients
            pdf1 = norm.pdf(xb1)
            pdf0 = norm.pdf(xb0)
            grad = pdf1 * exog1 - pdf0 * exog0
            var_ME = np.dot(grad, np.dot(cov_params, grad))
            se_ME = np.sqrt(var_ME)
            # Compute z-value and p-value
            z = me / se_ME
            p_value = 2 * (1 - norm.cdf(abs(z)))
            return me, se_ME, p_value, grad

        # Compute marginal effects over centuries
        centuries = ['16', '17', '18']

        me_dict = {}
        var_dict = {}
        grad_dict = {}

        f.write("\nAverage Marginal Effects:\n")
        f.write(f"{'Century':>10} {'dy/dx':>10} {'Std. Err.':>12} {'z':>8} {'P>|z|':>8} {'[95% Conf. Interval]':>22}\n")
        for c in centuries:
            me, se_ME, p_value, grad = compute_marginal_effect(c)
            me_dict[c] = me
            var_ME = se_ME**2
            var_dict[c] = var_ME
            grad_dict[c] = grad
            conf_int_low = me - 1.96 * se_ME
            conf_int_high = me + 1.96 * se_ME
            z = me / se_ME
            f.write(f"{c:>10} {me:>10.7f} {se_ME:>12.7f} {z:>8.2f} {p_value:>8.4f} [{conf_int_low:.7f}, {conf_int_high:.7f}]\n")
        f.write("\n")

        # Perform tests of equality of marginal effects
        f.write("Tests of Equality of Marginal Effects:\n")
        # Test between century 16 and 17
        dME = me_dict['16'] - me_dict['17']
        Var_dME = var_dict['16'] + var_dict['17'] - 2 * np.dot(grad_dict['16'], np.dot(cov_params, grad_dict['17']))
        se_dME = np.sqrt(Var_dME)
        z = dME / se_dME
        p_value = 2 * (1 - norm.cdf(abs(z)))
        f.write(f"Test ME at century 16 vs 17: chi2(1) = {z**2:.2f}, P>|chi2| = {p_value:.4f}\n")

        # Test between century 16 and 18
        dME = me_dict['16'] - me_dict['18']
        Var_dME = var_dict['16'] + var_dict['18'] - 2 * np.dot(grad_dict['16'], np.dot(cov_params, grad_dict['18']))
        se_dME = np.sqrt(Var_dME)
        z = dME / se_dME
        p_value = 2 * (1 - norm.cdf(abs(z)))
        f.write(f"Test ME at century 16 vs 18: chi2(1) = {z**2:.2f}, P>|chi2| = {p_value:.4f}\n")

        # Test between century 17 and 18
        dME = me_dict['17'] - me_dict['18']
        Var_dME = var_dict['17'] + var_dict['18'] - 2 * np.dot(grad_dict['17'], np.dot(cov_params, grad_dict['18']))
        se_dME = np.sqrt(Var_dME)
        z = dME / se_dME
        p_value = 2 * (1 - norm.cdf(abs(z)))
        f.write(f"Test ME at century 17 vs 18: chi2(1) = {z**2:.2f}, P>|chi2| = {p_value:.4f}\n")
        f.write("\n")

        # Compute summary statistics of the dependent variable
        mean_dep_var = df[dep_var].mean()
        std_dep_var = df[dep_var].std()
        min_dep_var = df[dep_var].min()
        max_dep_var = df[dep_var].max()
        n_obs = df[dep_var].count()

        f.write(f"Summary Statistics for '{dep_var}':\n")
        f.write(f"{'Variable':<15} {'Obs':>10} {'Mean':>10} {'Std. Dev.':>12} {'Min':>8} {'Max':>8}\n")
        f.write(f"{dep_var:<15} {n_obs:>10} {mean_dep_var:>10.7f} {std_dep_var:>12.7f} {min_dep_var:>8} {max_dep_var:>8}\n")
        f.write("\n")
        f.write("-" * 80)
        f.write("\n")

/Users/niclasgriesshaber/anaconda3/envs/guilds-llm-env/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/niclasgriesshaber/anaconda3/envs/guilds-llm-env/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
